In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re
from shapely.geometry import Point, Polygon

##### Prep

Carga de datos

In [ ]:
path_file = '../data/bitacoras_corte_julio.csv' 

In [ ]:
relacion_tecnicos  = pd.read_csv('../data/tecnicos_programa.csv')

In [ ]:
relacion_tecnicos.info()

In [ ]:
relacion_tecnicos['Región de atención'].unique()

In [ ]:
relacion_tecnicos['Región de atención'].replace({'Sierra de Flores Magon ':'Cañada',
                                                 'sierra de Flores Magon ':'Cañada',
                                                 'Sierra Sur_Sola de Vega':'Sierra Sur',
                                                 'Costa ':'Costa'
                                                 },
                                        inplace=True)

In [ ]:
relacion_tecnicos['Región de atención'].unique()

In [ ]:
#nombre tecnicos
resumen_bita = pd.read_csv('../data/resumen_bita.csv')

In [ ]:
# resumen_bita.iloc[:,[25,27,40,41,42,43,44,45,46,47,48,49]].tail(3)

In [ ]:
#relacion de programa con tecnicos
usuarios = pd.read_csv('../data/usuarios_3.csv')

In [ ]:
usuarios.info()

In [ ]:
usuarios.head(2)

In [ ]:
usuarios.shape

In [ ]:
usuarios['id'].nunique()


In [ ]:
usuarios['user_name'].nunique()

In [ ]:
resumen_bita.info()

In [ ]:
resumen_bita=resumen_bita[resumen_bita.Proyecto =='SEFADER']

In [ ]:
resumen_bita.shape

In [ ]:
resumen_bita[['ID de bitacora','Tipo de superficie','Tipo de bitacora']].isnull().sum()

In [ ]:
resumen_bita[['deleted_BitacoraAgronomica','deleted_Sitios','deleted_RegistroDeParcelas','deleted link parcela-bitacora']].sum()

In [ ]:
resumen_bita = resumen_bita[(resumen_bita['deleted_BitacoraAgronomica'] < 1)]

In [ ]:
resumen_bita.shape

In [ ]:
resumen_bita = resumen_bita[(resumen_bita['deleted_RegistroDeParcelas'] < 1)]

In [ ]:
resumen_bita.shape

In [ ]:
resumen_bita = resumen_bita[(resumen_bita['deleted link parcela-bitacora'] < 1)]

In [ ]:
resumen_bita.shape

In [ ]:
resumen_bita = resumen_bita[(resumen_bita['deleted_Sitios'] < 1)]

In [ ]:
resumen_bita.shape

In [ ]:
resumen_bita['Tipo de superficie'].isnull().sum()

In [ ]:
resumen_bita.groupby('Tipo de bitacora')['Tipo de superficie'].apply(lambda grupo: grupo.isnull().sum())

In [ ]:
#agregar valor de innovacion 
label_inn={
    'AE':'Innovacion',
    'AI':'Innovacion'
}

In [ ]:
resumen_bita['Tipo de superficie'] = resumen_bita['Tipo de superficie'].fillna(resumen_bita['Tipo de bitacora'].map(label_inn))

In [ ]:
resumen_bita.groupby('Tipo de bitacora')['Tipo de superficie'].apply(lambda grupo: grupo.isnull().sum())

In [ ]:
modulos_nulos=resumen_bita[resumen_bita['Tipo de superficie'].isnull()]

In [ ]:
modulos_nulos.to_csv('./modulos_sin_tipo_superficie.csv')

In [ ]:
resumen_bita.shape

In [ ]:
resumen_bita.dropna(subset=['Tipo de superficie'],inplace=True)

In [ ]:
# -32 de modulos
resumen_bita.shape

In [ ]:
resumen_bita['id'] = resumen_bita['ID de bitacora']+"_"+resumen_bita['Tipo de bitacora'] + "_"+resumen_bita['Tipo de superficie']

In [ ]:
resumen_bita['id'].nunique()

In [ ]:
resumen_bita.to_csv('./resumen_repetidos_ids.csv')

In [ ]:
#aun despues de ver las bitacoras, algunas se repetien, se decide conservar el primero porque usualmente contiene informacion completa o correcta
resumen_bita.drop_duplicates(subset=['id'],keep='first',inplace=True)

In [ ]:
resumen_bita['Tipo de superficie'].unique()

In [ ]:
resumen_bita['Tipo de bitacora'].unique()

In [ ]:
resumen_bita.shape

In [ ]:
area_i = pd.read_csv('../data/AI_completo_todosCiclos - Copy.txt',sep='\t')

In [ ]:
area_i = area_i[area_i['Proyecto']=='SEFADER']

In [ ]:
area_i.info()

In [ ]:
area_i['Año'].unique()

In [ ]:
area_i = area_i[area_i['deleted_AreasDeImpacto'] < 1]

In [ ]:
area_i.shape

In [ ]:
area_i['id'] = area_i['ID Área de Impacto']+'_'+'AI simplificada'+'_'+'Innovacion'

In [ ]:
area_i['id'].nunique()

In [ ]:
resumen_bita = resumen_bita[['Bitacora registrada por','ID creado por','id']]

In [ ]:
area_i = area_i[['Nombre de usuario del sistema','Creado_Id usuario','id']]

In [ ]:
area_i.rename(columns={'Creado_Id usuario':'ID creado por','Nombre de usuario del sistema':'Bitacora registrada por'},inplace=True)

In [ ]:
resumen_bita.shape

In [ ]:
area_i.shape

In [ ]:
union_sets = pd.concat([resumen_bita, area_i], ignore_index=True)

In [ ]:
union_sets.shape

In [ ]:
corte = pd.read_csv(path_file)

In [ ]:
# 1 colocar nombre del tecnico y programa
corte.info()

In [ ]:
corte['Tipo.de.bitacora'].unique()

In [ ]:
corte[['ID.de.bitacora','Tipo.de.superficie','Tipo.de.bitacora']].isnull().sum()

In [ ]:
corte['Tipo.de.superficie'].unique()

In [ ]:
corte['id'] = corte['ID.de.bitacora']+'_'+corte['Tipo.de.bitacora']+'_'+corte['Tipo.de.superficie']

In [ ]:
corte['id'].nunique()

In [ ]:
corte_plus=corte.merge(union_sets[['Bitacora registrada por','ID creado por','id']],
            on='id',
            how='left'
            )

In [ ]:
corte_plus.shape

In [ ]:
corte_plus=corte_plus.merge(usuarios[['id','user_name','first_name','last_name']],
                    left_on='ID creado por',right_on='id',how='left')

In [ ]:
corte_plus.shape

In [ ]:
corte_plus['ID creado por'].isnull().sum()

In [ ]:
nulos_nombres_tecnicos=corte_plus[['Bitacora.registrada.por','Bitacora registrada por','ID creado por']][corte_plus['ID creado por'].isnull()]

In [ ]:
nulos_nombres_tecnicos

In [ ]:
nulos_nombres_tecnicos.to_csv('./tec_no_identificados.csv')

# Comienza corte sept

In [2]:
corte_sept = pd.read_csv('../data/corte_30_sept.csv')

/tmp/ipykernel_7508/494661461.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  corte_sept = pd.read_csv('../data/corte_30_sept.csv')


In [4]:
corte_sept.shape

(42873, 79)

In [5]:
corte_sept.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42873 entries, 0 to 42872
Data columns (total 79 columns):
 #   Column                                                                                             Non-Null Count  Dtype  
---  ------                                                                                             --------------  -----  
 0   ID.de.bitacora                                                                                     42873 non-null  object 
 1   Tipo.de.superficie                                                                                 42873 non-null  object 
 2   ID.del.productor                                                                                   4144 non-null   object 
 3   Nombre.Productor                                                                                   42873 non-null  object 
 4   CURP                                                                                               4144 non-null   obj

In [6]:
corte_sept['ID.de.bitacora'].nunique()

42484

In [7]:
corte_sept.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42873 entries, 0 to 42872
Data columns (total 79 columns):
 #   Column                                                                                             Non-Null Count  Dtype  
---  ------                                                                                             --------------  -----  
 0   ID.de.bitacora                                                                                     42873 non-null  object 
 1   Tipo.de.superficie                                                                                 42873 non-null  object 
 2   ID.del.productor                                                                                   4144 non-null   object 
 3   Nombre.Productor                                                                                   42873 non-null  object 
 4   CURP                                                                                               4144 non-null   obj

In [8]:
corte_sept['Tipo.de.superficie'].isnull().sum()

0

In [9]:
corte_sept['primary_k'] = corte_sept['ID.de.bitacora']+'_'+corte_sept['Tipo.de.bitacora']+'_'+corte_sept['Tipo.de.superficie']

In [10]:
corte_sept['primary_k'].nunique()
#42873

42873

In [11]:
#union de nombre tecnioc para determinar programa
programa = pd.read_csv('../data/relacion_tecnicos_programa_eagrology.csv')

In [12]:
programa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               241 non-null    int64 
 1   Bitacora.registrada.por  241 non-null    object
 2   NOMBRE DEL USUARIO       230 non-null    object
 3   Nombre del técnico       177 non-null    object
 4   Programa                 177 non-null    object
 5   Región de atención       177 non-null    object
dtypes: int64(1), object(5)
memory usage: 11.4+ KB


In [13]:
programa['Bitacora.registrada.por'].nunique()
#241

241

In [14]:
sept=corte_sept.merge(programa[['Bitacora.registrada.por','Programa']],on='Bitacora.registrada.por',how='left')

In [15]:
sept.shape

(42873, 81)

In [16]:
sept.Programa.unique()

array([nan, 'Abasto Seguro de Maíz', 'Autosuficiencia Alimentaria'],
      dtype=object)

In [17]:
sept.Programa.fillna('Sin Identificar',inplace=True)

/tmp/ipykernel_7508/4004509374.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sept.Programa.fillna('Sin Identificar',inplace=True)


In [18]:
sept.Programa.unique()

array(['Sin Identificar', 'Abasto Seguro de Maíz',
       'Autosuficiencia Alimentaria'], dtype=object)

In [19]:
sept['Anio.Bitacora'].unique()

array(['PV_2023', 'OI_2023_2024', 'OI_2022_2023', 'PV_2024', 'PV_2022',
       nan, 'PV_2019', 'PV_2020', 'PV_2021', 'OI_2021_2022',
       'OI_2018_2019', 'OI_2024_2025', 'PV_2025'], dtype=object)

In [20]:
sept['Anio.Bitacora'].fillna('Vacío',inplace=True)

/tmp/ipykernel_7508/1856748992.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sept['Anio.Bitacora'].fillna('Vacío',inplace=True)


In [21]:
sept['Anio.Bitacora'].unique()

array(['PV_2023', 'OI_2023_2024', 'OI_2022_2023', 'PV_2024', 'PV_2022',
       'Vacío', 'PV_2019', 'PV_2020', 'PV_2021', 'OI_2021_2022',
       'OI_2018_2019', 'OI_2024_2025', 'PV_2025'], dtype=object)

In [22]:
def replace_chracter(x):
    text = re.sub('_',' ',x)
    return text

In [23]:
sept['Anio.Bitacora'] = sept['Anio.Bitacora'].apply(replace_chracter)

In [24]:
sept['Anio.Bitacora'].unique()

array(['PV 2023', 'OI 2023 2024', 'OI 2022 2023', 'PV 2024', 'PV 2022',
       'Vacío', 'PV 2019', 'PV 2020', 'PV 2021', 'OI 2021 2022',
       'OI 2018 2019', 'OI 2024 2025', 'PV 2025'], dtype=object)

In [25]:
# %%timeit
def to_float(i):
    """
    el parámetro i es un del tipo str
    -----------------------------------
    La función to_float convierte a 'i' a tipo flotante
    -----------------------------------
    devuelve un valor del tipo float
    
    """
    try:
        i = re.sub(r"\\t",'',i)
        i = float(i)
        return i 
    except:
        pass
    return i

In [26]:
# %%timeit
def transform(row):
    """
    el parámetro row es un valor del tipo str
    ----------------------------------------------
    separa los elementos de cada row de acuerdo a ciertos caracteres
    depura aquellos que no cumplen ciertas condiciones asignandoles el valor de lista vacia
    transforma cada cadena de texto a valores del tipo float
    ------------------------------------------------
    Retorna coor la cual puede ser una lista con valores flotantes 
    o una lista vacía.

    """
    input = row.split(',')

    size_list = len(input)


    if size_list ==1:
        coor = None
    elif size_list ==2:
        try:
            # input.replace('\\t','')
            pre_punto = map(to_float,input)
            coor = list(pre_punto)
            if None in coor:
                coor = None
                return coor
            else:
                return coor
        except ValueError as e:
            print(e)
            return None
    elif size_list >2:
        not_point = re.split(r",0\\n\\n|,0|0\s-|,",row)
        if ("" in not_point):
            not_point.remove("")
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
            return coor
        else:
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
            return coor

In [27]:
# %%timeit
def to_matrix(listado):
    """
    el parámetro listado es un objeto del tipo lista
    -----------------------------------
    transforma una lista a un array de numpy
    asigna la forma del array
    -----------------------------------
    devuelve una matriz
    """
    n = len(listado)
    c = 2
    f =  n/c
    f = int(f)
    vector = np.array(listado)
    matriz = vector.reshape(f,c)
    return matriz

In [28]:
# %%timeit
def geometry_apply(x):
    """
    el parámetro x es una lista
    ------------------------------------------
    de acuerdo a la longitud de la lista aplica una o dos funciones
    -----------------------------------------
    devuelve geometry que puede ser un objeto Point, Polygon o un valor None
    """

    n = x
    if len(n) == 2:
        # orden=arreglo(n)
        # p=sorted(n)
        # print(p)
        geometry = Point(n)
    elif len(n) >5 and len(n) %2 == 0: #Nueva validación
        listado = to_matrix(n)
        geometry = Polygon(listado)
    else:
        geometry = None
    print(geometry)
    return geometry

In [29]:
# %%timeit
def typeshape(x):
    """
    El parámetro x es un str
    ----------------------------------------
    De acuerdo al valor genera ciertas cadenas de texto
    ---------------------------------------
    retorna label que es un str
    """
    n=x
    if int(n) == 2:
        label = "Point"
    else:
        label = "Polygon"
    return label  

In [30]:
# %%timeit
def to_geopandas_df(df,origin_file):
    """
    file_input recibe al archivo Sitios.txt
    -----------------------------------------------
    Lee el archivo
    genera columnas y aplica funciones
    crea 3 subsets distintos
    elimina columnas temporales y columna Coordenadas areas
    exportar los sets a un formato específico
    -----------------------------------------------
    genera los archivos de salida o un error
    """
    try:
        
         #read   
        # sitios= pd.read_csv(file_input)
        sitiosbd = df.copy()
        #crear col geografica
        sitiosbd['info_geo'] = sitiosbd['Coordenadas.areas']
        #castear dato a str
        sitiosbd['info_geo']=sitiosbd['info_geo'].astype(str)
        #se aplica transform
        sitiosbd['info_geo']= sitiosbd['info_geo'].apply(transform)
        desechados=sitiosbd[sitiosbd['info_geo'].isnull()]
        sitiosbd_depurate=sitiosbd[sitiosbd['info_geo'].notna()]
        sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))
        # sitiosbd_depurate.to_csv(f'{PATH}n_vertices.csv')
        # sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(arreglo)
        sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)
        sitiosbd_depurate=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].notna()]
        desechados_2=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].isnull()]
        desechados = pd.concat([desechados_2,desechados],axis=0)
        desechados.to_csv(f'./errors_{origin_file}.csv', index=False)
        sitiosbd_depurate['type_shape'] = sitiosbd_depurate['n_vertices'].apply(typeshape)
        sitiosbd_polygon = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Polygon']
        sitiosbd_point = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Point']
        
        gdf_points_sitios = gpd.GeoDataFrame(sitiosbd_point,geometry='type_geometry',crs="EPSG:4326")
        gdf_polygons_sitios = gpd.GeoDataFrame(sitiosbd_polygon,geometry='type_geometry',crs="EPSG:4326")
 #el formato GeoJSON no soporta el formato de listas(un arreglo)
        # geopandas_df  = gpd.GeoDataFrame(sitiosbd_depurate,geometry='type_geometry',crs="EPSG:4326")
        # eliminar = ['Coordenadas.areas','info_geo','n_vertices','Nombre.Productor','Bitacora.registrada.por']
        # eliminar = ['Coordenadas.areas','info_geo','n_vertices','type_shape','Nombre.Productor','Bitacora.registrada.por']
        eliminar = ['Coordenadas.areas','info_geo','n_vertices','type_shape','Bitacora.registrada.por','Nombre.Productor']
        # geopandas_df=geopandas_df.drop(columns=eliminar)
        gdf_points_sitios=gdf_points_sitios.drop(columns=eliminar)
        gdf_polygons_sitios=gdf_polygons_sitios.drop(columns=eliminar)
        return gdf_points_sitios,gdf_polygons_sitios
        # gdf_points_sitios.to_file(puntos,driver='GeoJSON')
        # gdf_polygons_sitios.to_file(poligonos,driver='GeoJSON')
    except FileNotFoundError as e:
        print(f'Por favor ejecute el script en el mismo directorio donde se encuentre el archivo Sitio.txt {e}')

Transf to geo

In [31]:
pts, pols=to_geopandas_df(sept,'sept_30_')

/tmp/ipykernel_7508/923546810.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))


POLYGON ((-95.89601519 17.75389612, -95.89536609 17.75461648, -95.89739921 17.75523465, -95.89780154 17.75434059, -95.89601519 17.75389612))
POLYGON ((-95.51424973 16.4406485, -95.51435165 16.44067422, -95.51416926 16.44113214, -95.51419072 16.44124018, -95.5143141 16.4413225, -95.5144643 16.44130707, -95.51460378 16.44123504, -95.5144804 16.44155403, -95.5143141 16.44169809, -95.51417462 16.4417444, -95.51405124 16.44174954, -95.51396541 16.44171867, -95.5139064 16.44167237, -95.51386885 16.44161063, -95.51374011 16.4416878, -95.51377229 16.44201194, -95.51360063 16.44251616, -95.51324121 16.44245956, -95.5132734 16.44200165, -95.51322512 16.44187817, -95.51321439 16.44175983, -95.51325731 16.44161063, -95.51333777 16.44145628, -95.51348261 16.44132765, -95.51367037 16.4411733, -95.51385812 16.44106011, -95.51401369 16.44096749, -95.51417999 16.44063821, -95.51424973 16.4406485))
POLYGON ((-95.52305923 16.44835046, -95.52331136 16.44731633, -95.52385317 16.44749126, -95.52348302 16.44

/tmp/ipykernel_7508/923546810.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)


In [32]:
pts.shape

(38729, 79)

In [33]:
pols.shape

(3468, 79)

Carga de layers

In [34]:
# mun_oax = gpd.GeoDataFrame(gpd.read_file('../data/regiones_mun_oax.geojson',geometry='geometry',crs="EPSG:4326"))
reg_oax = gpd.GeoDataFrame(gpd.read_file('../data/regiones_oax.geojson',geometry='geometry',crs="EPSG:4326"))

/home/ubuntu/sefader/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver GeoJSON does not support open option GEOMETRY
  return ogr_read(
/home/ubuntu/sefader/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver GeoJSON does not support open option CRS
  return ogr_read(
/home/ubuntu/sefader/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver GeoJSON does not support open option GEOMETRY
  return ogr_read(
/home/ubuntu/sefader/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver GeoJSON does not support open option CRS
  return ogr_read(


In [35]:
ds = gpd.sjoin(pols,reg_oax, how="inner", predicate='within')

In [36]:
ds.shape

(3407, 86)

In [37]:
pts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 38729 entries, 4 to 40922
Data columns (total 79 columns):
 #   Column                                                                                             Non-Null Count  Dtype   
---  ------                                                                                             --------------  -----   
 0   ID.de.bitacora                                                                                     38729 non-null  object  
 1   Tipo.de.superficie                                                                                 38729 non-null  object  
 2   ID.del.productor                                                                                   1789 non-null   object  
 3   CURP                                                                                               1789 non-null   object  
 4   ID.parcela                                                                                         0 non-null

In [38]:
# Función para intercambiar el orden de latitud y longitud en cada Point
def swap_lat_lon(point):
    return Point(point.y, point.x)

# Aplica la función a cada fila de la columna 'geometry' para corregir el orden


In [39]:
pts['type_geometry'] = pts['type_geometry'].apply(swap_lat_lon)

In [40]:
ds2 = gpd.sjoin(pts,reg_oax, how="inner", predicate='within')

In [41]:
ds2.shape

(38049, 86)

In [ ]:
# ds2.explore()

In [42]:
ds2.shape

(38049, 86)

In [ ]:
# pts

In [ ]:
# ds2

In [43]:
ds.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3407 entries, 2 to 42872
Data columns (total 86 columns):
 #   Column                                                                                             Non-Null Count  Dtype   
---  ------                                                                                             --------------  -----   
 0   ID.de.bitacora                                                                                     3407 non-null   object  
 1   Tipo.de.superficie                                                                                 3407 non-null   object  
 2   ID.del.productor                                                                                   2240 non-null   object  
 3   CURP                                                                                               2240 non-null   object  
 4   ID.parcela                                                                                         1930 non-nu

In [44]:
ds2.columns

Index(['ID.de.bitacora', 'Tipo.de.superficie', 'ID.del.productor', 'CURP',
       'ID.parcela', 'ID.del.sitio', 'Nombre.de.la.Parcela', 'Estado.areas',
       'Municipio.areas', 'Tipo.de.bitacora', 'Anio.Bitacora', 'Ciclo',
       'Regimen.hidrico', 'Superficie.sembrada',
       'Superficie.Total.de.la.parcela', 'Cultivo.1', 'Cultivo.2', 'Cultivo.3',
       'Proyecto', 'NOMBRE DEL USUARIO', 'Id_Cultivos',
       '1_AC_Cultivos_de_cobertura',
       '2_AC_Fertilizadora_reformadora_aplicada_para_poder_hacer_AC',
       '3_AC_Implementos_para_acondicionamiento_de_parcela_anio_cero',
       '4_AC_Implementos_para_adaptar_maquinaria_convencional_para_uso_en_AC',
       '5_AC_Inclusion_de_nuevos_cultivos_en_sistema_de_produccion_de_maiz_y_grano_pequenio',
       '6_AC_Labranza_en_lineas_de_Siembra',
       '7_AC_Manejo_de_rastrojo_uso_de_desmenuzadora_manual_o_mecanica',
       '8_AC_Maquinaria_en_AC_para_operaciones_en_practicas_de_labranza_cero_o_reducido',
       '9_AC_Rotacion_Diversific

In [45]:
ds2['Region'].unique()

array(['Istmo', 'Sierra Sur', 'Cañada', 'Valles centrales', 'Mixteca',
       'Sierra Norte', 'Papaloapan', 'Costa'], dtype=object)

##### Formateo

In [46]:
ds['Region'].replace({'Valles centrales':'Valles Centrales'},inplace=True)

/tmp/ipykernel_7508/3491809979.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ds['Region'].replace({'Valles centrales':'Valles Centrales'},inplace=True)


In [47]:
ds['Region'].unique()

array(['Istmo', 'Sierra Sur', 'Papaloapan', 'Costa', 'Cañada',
       'Valles Centrales', 'Mixteca', 'Sierra Norte'], dtype=object)

In [48]:
ds2['Region'].replace({'Valles centrales':'Valles Centrales'},inplace=True)

/tmp/ipykernel_7508/2739909976.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ds2['Region'].replace({'Valles centrales':'Valles Centrales'},inplace=True)


In [49]:
ds2['Region'].unique()

array(['Istmo', 'Sierra Sur', 'Cañada', 'Valles Centrales', 'Mixteca',
       'Sierra Norte', 'Papaloapan', 'Costa'], dtype=object)

In [51]:
ds['Programa'].unique()

array(['Sin Identificar', 'Abasto Seguro de Maíz',
       'Autosuficiencia Alimentaria'], dtype=object)

In [52]:
dataframe = pd.concat([ds,ds2],ignore_index=True)

In [53]:
ds2.to_file('../data/puntos_sept.geojson', driver='GeoJSON')
ds.to_file('../data/poligonos_sept.geojson', driver='GeoJSON')


In [54]:
dataframe.to_csv('../data/corte_clean_sept.csv')

In [56]:
dataframe['ID.del.productor'].nunique()

3352

In [62]:
dataframe['index_right'].head(10)

0    2
1    2
2    6
3    4
4    4
5    4
6    1
7    0
8    7
9    7
Name: index_right, dtype: int64

##### Carga datos Aux

In [ ]:
usuarios.head(2)

#### KPI - Sumarizacion

In [ ]:
# Municipios por región
ds.groupby('Region')['Municipio.areas'].nunique()

In [ ]:
# por región
ds2.groupby('Region')['Municipio.areas'].nunique()

In [ ]:
#posible salida para consumo
pd.DataFrame(ds2.groupby('Region')['Municipio.areas'].nunique()).reset_index()

In [ ]:
# numero de farmers por región
pd.DataFrame(
(ds
 .assign(farmers_unicos = ds['ID.del.productor'] + ds['Nombre.Productor'] + ds['CURP'])
 .groupby('Region')['farmers_unicos'].nunique()
#  .farmers_unicos.nunique()
    # farmers = union_fields.nunique()
)).reset_index()



In [ ]:
#superficie por región
#Posible salida 2
dict(ds.groupby('Region')['Superficie.sembrada'].sum())


In [ ]:
#parcelas por región
dict(ds.groupby('Region')['type_geometry'].nunique())

In [ ]:
#bitacoras por región
dict(ds.groupby('Region')['ID.de.bitacora'].nunique())

Por tipo de parcela

In [ ]:
ds.groupby('Tipo.de.bitacora')['Superficie.sembrada'].sum().reset_index()

In [ ]:
ds['Tipo.de.bitacora'].replace({'AE':'Área de extensión',
                                'AI':'Área de impacto',
                                'AI simplificada':'Área de impacto',
                                'MODULO':'Módulo'},inplace=True)

In [ ]:
ds.groupby('Tipo.de.bitacora')['Superficie.sembrada'].sum().reset_index()

Por ciclo Productivo

In [ ]:
dict(ds.groupby('Anio.Bitacora')['Superficie.sembrada'].sum())
#reemplazar guines bajos por espacios

Por regimén hidríco

In [ ]:
ds.groupby('Regimen.hidrico')['Superficie.sembrada'].sum().reset_index()

In [ ]:
ds.groupby('Regimen.hidrico')['Tipo.de.bitacora'].count().reset_index()

##### test section

In [ ]:
# catalogo de desplegables

In [ ]:
list(ds['Anio.Bitacora'].unique())

In [ ]:
list(ds['Region'].unique())

In [ ]:
cat_reg=list(ds['Regimen.hidrico'].unique())

In [ ]:
cat_reg.insert(0,'TODOS')

In [ ]:
cat_reg

In [ ]:
cols = ['Anio.Bitacora','Regimen.hidrico','Region']
options = [['OI_2023_2024', 'PV_2024'],'RIEGO',[ 'Istmo','Mixteca','Costa']]

In [ ]:
list(dict(zip(cols, options)).items())

In [ ]:

def subset(dataset:str, columns_list:list,conditions_list:list) -> list:
    main_diccionary = dict(zip(columns_list,conditions_list))
    ops = [(dataset[llave]==val) for llave,val in list(main_diccionary.items())]
    
    print(ops)
    # return dataset

In [ ]:
subset('name',cols, options)

In [ ]:
listas = pd.read_csv('../data/Listas.csv')

In [ ]:
listas = listas.iloc[:,0:3]

In [ ]:
listas.head(2)

In [ ]:
listas.Lista.unique()

In [ ]:
listado_mun = listas[listas.Lista=='Municipios_mx_list'].reset_index(drop=True)

In [ ]:
listado_mun = listado_mun.iloc[1:,:]

In [ ]:
listado_mun.head(10)

In [ ]:
mun_oax = gpd.GeoDataFrame(gpd.read_file('../data/regiones_oax.geojson',geometry='geometry'))

In [ ]:
mun_oax.info()

In [ ]:
mun_oax

In [ ]:
file_polygons = '../data/poligonos.geojson'

In [ ]:
poli.geometry.iloc[0]

In [ ]:
poli = gpd.GeoDataFrame(gpd.read_file(file_polygons,geometry='geometry'))

In [ ]:
# poli.crs

In [ ]:
# poli.assign(area= poli.geometry.to_crs(epsg=4326))

In [ ]:
# poli.area.head(2)

In [ ]:
# pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# poli.area /10e4

In [ ]:
puntos = gpd.GeoDataFrame(gpd.read_file('../data/puntos.geojson',geometry= 'geometry'))

In [ ]:
puntos["Tipo.de.bitacora"].nunique()

In [ ]:
puntos.geometry.y